Imports

In [ ]:
import f90nml
import numpy as np
from pint import UnitRegistry; AssignQuantity = UnitRegistry().Quantity
import os
import reference_solution as refsol
from scipy.fft import rfft
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import icepinn as ip

torch.set_default_dtype(torch.float64)
print(torch.cuda.device_count())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

device = ip.get_device()

In [ ]:
# Read in GI parameters
inputfile = "GI parameters - Reference limit cycle (for testing).nml"
GI=f90nml.read(inputfile)['GI']
nx_crystal = GI['nx_crystal']
L = GI['L']
NBAR = GI['Nbar']
NSTAR = GI['Nstar']

# Define t range (needs to be same as training file)
RUNTIME = 5
NUM_T_STEPS = RUNTIME + 1
#NUM_T_STEPS = RUNTIME*5 + 1

# Define initial conditions
Ntot_init = np.ones(nx_crystal)
Nqll_init = ip.get_Nqll(Ntot_init)

# Define x, t pairs for training
X_QLC = np.linspace(-L,L,nx_crystal)
t_points = np.linspace(0, RUNTIME, NUM_T_STEPS)
x, t = np.meshgrid(X_QLC, t_points)
training_set = torch.tensor(np.column_stack((x.flatten(), t.flatten()))).to(device)

## Naming Legend

CL = curriculum learning  
SF = SF_Pinn architecture  
HE = hard-enforced initial condition  
{n}wide = nodes per FC-layer  
nodiff = diffusion term is excluded  
LBFGS = LBFGS was used


In [ ]:
MODEL_NAME = "HE_32wide_nodiff_LBFGSonly"

# Curriculum learning?
curriculum_learning = False
# Pre-load non-IC-enforced model? (CR only)
preload = False
# Use L-BFGS after Adam?
LBFGS = True

# Define model attributes
model_dimensions = torch.tensor([8, 32]).to(device)
is_sf_PINN = torch.tensor(False)
diffusion = False

In [ ]:
# instantiate model
model = ip.IcePINN(
	num_hidden_layers=model_dimensions[0], 
	hidden_layer_size=model_dimensions[1],
	is_sf_PINN=is_sf_PINN.item()).to(device)

# Attach model attributes as buffers so they can be saved and loaded
model.register_buffer('dimensions', model_dimensions)
model.register_buffer('is_sf_PINN', is_sf_PINN)

# Initialize model weights with HE initialization
model.apply(ip.init_HE)

# # Define learning rate scheduling scheme
# scheduler_summed = optim.lr_scheduler.ReduceLROnPlateau(
#         optimizer, mode='min', factor=0.5, patience=10000
#     )

In [ ]:
print(MODEL_NAME)
print(training_set.shape)
print(ip.calc_cp_loss(model, training_set, ip.get_misc_params()).shape)
print(ip.enforced_model(training_set, model).shape)
print(model)

Train model using AdamW

In [ ]:
if curriculum_learning:
    # First, train without IC enforced (if it wasn't pre-loaded)
    if not preload:
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
        ip.train_IcePINN(
            model=model, 
            optimizer=optimizer, 
            training_set=training_set, 
            epochs=100_000, 
            name=MODEL_NAME, 
            print_every=1_000,
            diffusion=diffusion,
            LR_scheduler=None,
            enforce_IC=False)
    else:
        model = ip.load_IcePINN(MODEL_NAME, pre_IC=True)
        model.train()


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

if curriculum_learning:
    # Gradually enforce IC over adjustment_period and keep it enforced
    ip.train_IcePINN(
        model=model, 
        optimizer=optimizer, 
        training_set=training_set, 
        epochs=200_000, 
        name=MODEL_NAME, 
        print_every=1_000,
        diffusion=diffusion,
        LR_scheduler=None,
        enforce_IC=True,
        adjustment_period=100_000)

else:
    # Train normally with IC enforced
    ip.train_IcePINN(
        model=model, 
        optimizer=optimizer, 
        training_set=training_set, 
        epochs=10, 
        name=MODEL_NAME, 
        print_every=1_000,
        diffusion=diffusion,
        LR_scheduler=None,
        enforce_IC=True,
        adjustment_period=0)

In [ ]:
# Load best model for future training
model = ip.load_IcePINN(MODEL_NAME)
model.train()

Prepare for L-BFGS optimization

In [ ]:
lbfgs_optim = torch.optim.LBFGS(
    params=model.parameters(), 
    lr=0.1,
    max_iter=5_000, 
    history_size=5_000
)
misc_params = ip.get_misc_params()
lbfgs_iter_counter = 0
lbfgs_print_freq = 1

# closure() is called by L-BFGS when you call step() up to max_iter times
def closure():
    #lbfgs_iter_counter += 1

    lbfgs_optim.zero_grad()
    loss = ip.calc_cp_loss(model, training_set, misc_params, diffusion)
    loss.backward(torch.ones_like(loss))

    #if lbfgs_iter_counter % lbfgs_print_freq == 0:
    # sum and print loss
    Ntot_loss = torch.sum(loss[:, 0]).item()
    Nqll_loss = torch.sum(loss[:, 1]).item()
    print(f"L-BFGS iteration loss: Ntot = {Ntot_loss:.3f}, Nqll = {Nqll_loss:.3f}")
    # {lbfgs_iter_counter}
    # Return as a summed scalar loss: required by L-BFGS
    return torch.add(Ntot_loss, Nqll_loss)

L-BFGS time

In [ ]:
if LBFGS:    
    # lbfgs_iter_counter = 0
    # lbfgs_print_freq = 1

    lbfgs_optim.step(closure)

    
    save_path = './models/'+MODEL_NAME

    # Create folder to store model in if necessary
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # Save model (not including initial condition wrapper)
    torch.save(model.state_dict(), save_path+'/post_LBFGS_params.pth')
    # TODO - update (load) so that it can load LBFGS results
    print("Done!")


Train some more if needed?

In [ ]:
# ip.train_IcePINN(
#     model=model, 
#     optimizer=optimizer, 
#     training_set=training_set, 
#     epochs=100_000, 
#     name=MODEL_NAME, 
#     print_every=1_000, 
#     diffusion=diffusion,
#     LR_scheduler=None,
#     enforce_IC=True,
#     adjustment_period=0)